In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

In [2]:
#  Import and read the nba data.
file_name = 'Resources/nba_2009_2011.csv'
df = pd.read_csv(file_name)
df.head()

,id,h_1st,h_2nd,h_3rd,h_4th,h_5th,h_OT_number,h_FinalPoints,h_o1st,h_o2nd,...,r_ORB,r_FT_FGA,r_ORt,r_Road,r_b2b,h_PointsWin,r_PointsWin,result,home_win,binary_home_win
0,0,23.2,19.6,25.2,23.9,2.8,0.2,94.7,22.2,24.6,...,26.88,0.2947,101.35,0.4,1,-0.7,4.5,hplost,False,0
1,1,20.4,21.1,23.9,24.5,0.0,0.0,89.9,26.0,25.9,...,24.85,0.2516,103.75,0.6,0,-10.4,2.6,hpwin,True,1
2,2,24.7,23.0,23.2,25.4,0.0,0.0,96.3,22.2,21.0,...,21.99,0.3126,109.04,0.5,1,3.3,4.0,hpwin,True,1
3,3,22.2,25.8,21.4,24.4,0.9,0.1,94.7,23.7,20.0,...,22.11,0.1889,106.61,0.4,0,4.5,0.2,hpwin,True,1
4,4,28.1,25.7,24.9,23.8,1.9,0.2,104.4,28.7,23.6,...,28.04,0.2246,110.42,0.7,0,-0.8,-1.6,hplost,True,1


In [3]:
#drop non-numerical data
drop_df = df.drop(columns=['result','home_win'], axis=1)
drop_df.head()

,id,h_1st,h_2nd,h_3rd,h_4th,h_5th,h_OT_number,h_FinalPoints,h_o1st,h_o2nd,...,r_eFG,r_TOV,r_ORB,r_FT_FGA,r_ORt,r_Road,r_b2b,h_PointsWin,r_PointsWin,binary_home_win
0,0,23.2,19.6,25.2,23.9,2.8,0.2,94.7,22.2,24.6,...,0.4683,15.85,26.88,0.2947,101.35,0.4,1,-0.7,4.5,0
1,1,20.4,21.1,23.9,24.5,0.0,0.0,89.9,26.0,25.9,...,0.4590,12.43,24.85,0.2516,103.75,0.6,0,-10.4,2.6,1
2,2,24.7,23.0,23.2,25.4,0.0,0.0,96.3,22.2,21.0,...,0.5467,16.74,21.99,0.3126,109.04,0.5,1,3.3,4.0,1
3,3,22.2,25.8,21.4,24.4,0.9,0.1,94.7,23.7,20.0,...,0.5093,13.11,22.11,0.1889,106.61,0.4,0,4.5,0.2,1
4,4,28.1,25.7,24.9,23.8,1.9,0.2,104.4,28.7,23.6,...,0.4931,10.71,28.04,0.2246,110.42,0.7,0,-0.8,-1.6,1


In [4]:
# split into features and target arrays
y = drop_df['binary_home_win'].values
X = drop_df.drop(['binary_home_win'], axis=1).values

In [5]:
# create standard scalar instance
scaler = StandardScaler()

In [6]:
#split preprocesed data into training/testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2)

In [7]:
#fit the standardscaler
X_scaler = scaler.fit(X_train)

In [8]:
#scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# define the model
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 60
hidden_nodes_layer3 = 5

nn = tf.keras.models.Sequential()

#first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

#output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#check structure
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               10600     
_________________________________________________________________
dense_1 (Dense)              (None, 60)                6060      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 305       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 16,971
Trainable params: 16,971
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
67/67 [==============================] - 0s 1ms/step - loss: 0.6800 - accuracy: 0.5980
Epoch 2/100
67/67 [==============================] - 0s 1ms/step - loss: 0.6626 - accuracy: 0.5980
Epoch 3/100
67/67 [==============================] - 0s 1ms/step - loss: 0.6508 - accuracy: 0.6037
Epoch 4/100
67/67 [==============================] - ETA: 0s - loss: 0.6407 - accuracy: 0.61 - 0s 1ms/step - loss: 0.6371 - accuracy: 0.6277
Epoch 5/100
67/67 [==============================] - 0s 1ms/step - loss: 0.6213 - accuracy: 0.6442
Epoch 6/100
67/67 [==============================] - 0s 1ms/step - loss: 0.6039 - accuracy: 0.6569
Epoch 7/100
67/67 [==============================] - 0s 1ms/step - loss: 0.5830 - accuracy: 0.6904
Epoch 8/100
67/67 [==============================] - 0s 1ms/step - loss: 0.5610 - accuracy: 0.7224
Epoch 9/100
67/67 [==============================] - 0s 1ms/step - loss: 0.5394 - accuracy: 0.7408
Epoch 10/100
67/67 [==============================] - 0s 952us/step

67/67 [==============================] - 0s 986us/step - loss: 0.0188 - accuracy: 0.9972
Epoch 82/100
67/67 [==============================] - 0s 1ms/step - loss: 0.0187 - accuracy: 0.9972
Epoch 83/100
67/67 [==============================] - 0s 960us/step - loss: 0.0186 - accuracy: 0.9972
Epoch 84/100
67/67 [==============================] - 0s 971us/step - loss: 0.0185 - accuracy: 0.9972
Epoch 85/100
67/67 [==============================] - 0s 933us/step - loss: 0.0184 - accuracy: 0.9972
Epoch 86/100
67/67 [==============================] - 0s 973us/step - loss: 0.0183 - accuracy: 0.9972
Epoch 87/100
67/67 [==============================] - 0s 930us/step - loss: 0.0183 - accuracy: 0.9972
Epoch 88/100
67/67 [==============================] - 0s 964us/step - loss: 0.0182 - accuracy: 0.9972
Epoch 89/100
67/67 [==============================] - 0s 960us/step - loss: 0.0181 - accuracy: 0.9972
Epoch 90/100
67/67 [==============================] - 0s 953us/step - loss: 0.0181 - accuracy: 0.

In [12]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

23/23 - 0s - loss: 2.4958 - accuracy: 0.4915
Loss: 2.495788812637329, Accuracy: 0.49152541160583496
